In [1]:
import glob, os
from os import listdir
from os.path import isfile, join
from astropy.io import fits
import numpy as np
from astropy.wcs import WCS
from FITS_tools.hcongrid import hcongrid
from astropy.nddata.utils import Cutout2D
from astropy.stats import sigma_clipped_stats
import scipy
from scipy import stats

# Functions

In [1]:
#creating a new background program for rectangle images

#maximo divisor comum
def mdc(m,n):
    div = min(m,n)
    big = max(m,n)
    remain = big % div
    while remain != 0:
        big = div
        div = remain
        remain = big % div
    return div

# retorna os valores de x e y divididos pelo mdc deles
def mdc_div(x,y):
    first = mdc(x,y)
    xn,yn = x/first,y/first
    if first != 1:
        return xn,yn

mdc_div(1024,1100)
#blablablablalblablalblablba

(256.0, 275.0)

In [3]:
user='usuario'

rawdir = '/home/'+user+'/Documents/Mestrado/FFIs/8_golden/teste/' #directory with the raw images
cdedir = '/home/'+user+'/dia_kepler/mychanges/Python/' #directory where the code 'lives'
clndir = '/home/'+user+'/Documents/Mestrado/FFIs/8_golden/teste/teste_clean/'#directory for the cleaned images to be output

#get the image list and the numberuction
os.chdir(rawdir) #changes to the raw image direcotory
files = [f for f in glob.glob("*.fits") if isfile(join(rawdir, f))] #gets the relevant files with the proper extension
files.sort()
nfiles = len(files)
os.chdir(cdedir) #changes back to the code directory

hdulist = fits.open(rawdir+files[0])
n_images = len(hdulist)-1

#if your matrix begins in zero:

# for j in range(1,len(hdulist)):
header = hdulist[1].header
w=WCS(header)
orgimg = hdulist[1].data

axs1, axs2 = np.shape(orgimg)
#se os axs1 e axs2 forem par, essa posição central será deslocada para uma a direita na linha e uma abaixo na col
#se ímpar sera no centro mesmo
cut = Cutout2D(orgimg,(int(axs2/2),int(axs1/2)),(axs1,axs2), wcs=w)
bigimg=cut.data
if np.min(bigimg) < 0:
    bigimg=bigimg+np.min(bigimg)
    add=1# variavel que eu vou ver se foi adicionado no final - lembrar dela

#creating the matrices that will be populate by the bck, sky values and etc
big_sig = np.zeros((axs1,axs2))
background = np.zeros((axs1,axs2))
mini_back = np.zeros((axs1,axs2))

In [19]:
teste= np.array([[1,2,1],[2,2,1],[1,1,2]])
teste1 = np.where(teste==1,teste,0)
teste2=np.where(teste==2,teste,0)
teste3=np.where(teste>1)
teste3

(array([0, 1, 1, 2]), array([1, 0, 1, 2]))

In [20]:
>>> mu, sigma = 0, 0.1 # mean and standard deviation
>>> s = np.random.normal(mu, sigma, 1000)
len(s)
mean,sky,std = scipy.stats.sigmaclip(s,low=2.0,high=2.0)
print(len(mean))
print(np.max(s),np.max(mean))
mean

920
0.2914862583831651 0.16827414165713692


array([-3.83027989e-02, -6.41729697e-03,  2.76373207e-02,  4.44398076e-02,
        9.12429058e-02, -1.25552804e-01,  1.53873045e-01,  8.88439435e-02,
        2.14485742e-02, -1.02487943e-02,  1.63046539e-03,  4.97887434e-02,
       -3.48697922e-03,  8.70236230e-02,  3.59304036e-02, -7.10142699e-02,
       -1.44240649e-02,  1.28660246e-01, -2.56453113e-02, -1.02592729e-01,
        9.02938698e-02, -8.29011971e-02, -8.01979774e-03, -2.84680589e-02,
        1.29164562e-01, -9.50884289e-02,  9.18328364e-03, -6.25914167e-02,
        1.21671968e-01,  8.75183847e-03,  3.63064084e-03, -6.59910006e-02,
        8.85110337e-02, -1.91516578e-02,  1.10377825e-01, -2.65481512e-02,
        4.15079599e-02,  1.39172250e-01, -1.10112257e-02, -1.55602397e-02,
       -1.30514032e-01, -8.03764615e-02, -5.58120009e-02, -9.01800517e-02,
        1.44070284e-01, -1.02140975e-01,  4.35458845e-02,  1.56594928e-01,
       -1.12241719e-01,  7.95511901e-02, -6.68340593e-02, -1.59840685e-02,
        1.35158381e-01,  

In [21]:
#comparing blocks of 256 X 275 and mini blocks of 64 X 55 
cut_line = int(mdc_div(1024,1100)[0])
cut_mini_line = int(cut_line/4)

cut_col = int(mdc_div(1024,1100)[1])
cut_mini_col = int(cut_col/5)

# astropy image

In [22]:
# for k in range(0,axs2,cut_col):
#     for l in range(0,axs1,cut_line):
#         img = bigimg[l:l+cut_line,k:k+cut_col]
        
#         #astropy routine - uses the median to define the center - slower:
#         mean, sky, sig = sigma_clipped_stats(img, sigma=2.5)
#         background[l:l+cut_line,k:k+cut_col] = sky
#         sigma[l:l+cut_line,k:k+cut_col] = sig
            
# for m in range(0,axs2,cut_mini_col):
#     for n in range(0,axs1,cut_mini_line):
#         mini_img = bigimg[n:n+cut_mini_line,m:m+cut_mini_col]
        
#         #astropy routine - uses the median to define the center - slower:
#         mmean, msky, msig = sigma_clipped_stats(mini_img, sigma = 2.5)
#         mini_back[n:n+cut_mini_line,m:m+cut_mini_col] = msky

# sub = bigimg - mini_back + background

# finme = "astropy.fits"       
# shd = fits.PrimaryHDU(sub, header=header)
# shd.writeto(clndir+finme,overwrite=True)

# scipy image

In [23]:
# for k in range(0,axs2,cut_col):
#     for l in range(0,axs1,cut_line):
#         #creating the small matrices
#         img = bigimg[l:l+cut_line,k:k+cut_col]
        
#         #scipy routine- uses the mean to define the center - faster:       
#         cimg, clow, chigh = scipy.stats.sigmaclip(img, low=2.5, high = 2.5) #do a 2.5 sigma clipping
#         sky = np.median(cimg) #determine the sky value
#         background[l:l+cut_line,k:k+cut_col] = sky
            
# for m in range(0,axs2,cut_mini_col):
#     for n in range(0,axs1,cut_mini_line):
#         mini_img = bigimg[n:n+cut_mini_line,m:m+cut_mini_col]
        
#         #scipy routine - uses the mean to define the center - faster:        
#         cc, cclow, cchigh = scipy.stats.sigmaclip(mini_img, low=2.5, high = 2.5) #do a 2.5 sigma clipping
#         msky = np.median(cc) #determine the sky value
#         mini_back[n:n+cut_mini_line,m:m+cut_mini_col] = msky

# sub = bigimg - mini_back + background

# finme = "scipy.fits"       
# chd = fits.PrimaryHDU(sub, header=header)
# chd.writeto(clndir+finme,overwrite=True)

In [24]:
# clndir = '/home/'+user+'/Documents/Mestrado/FFIs/8_golden/teste/teste_clean/'#directory for the cleaned images to be output

# #get the image list and the numberuction
# os.chdir(clndir) 

# original = "kplr2009114174833_ffi-cal.fits"
# astropy = "astropy.fits"
# scipy= "scipy.fits"
# ryan="ryan.fits"
# orig = fits.getdata(clndir+original)
# ast = fits.getdata(clndir+astropy)
# sci = fits.getdata(clndir+scipy)
# ry=fits.getdata(clndir+ryan)

# Astropy-orig

In [25]:
# a = ast-orig
# hdu = fits.PrimaryHDU(a)
# hdu.writeto(clndir+'ast-or',overwrite=True)

# Scipy-orig

In [26]:
# b = sci-orig
# hdv = fits.PrimaryHDU(b)
# hdv.writeto(clndir+'sci-or',overwrite=True)

# Scipy-astropy

In [27]:
# c = sci-ast
# hdw = fits.PrimaryHDU(c)
# hdw.writeto(clndir+'sci-ast',overwrite=True)

# Astropy-Scipy

In [28]:
# d = ast-sci
# hdx = fits.PrimaryHDU(d)
# hdx.writeto(clndir+'ast-sci',overwrite=True)

# Ryan-orig

In [29]:
# e= ry-orig[:,:1024]
# hdz=fits.PrimaryHDU(e)
# hdz.writeto(clndir+'ryan-orig',overwrite=True)

# Astropy-ryan

In [30]:
# f= ast[:,:1024]-ry
# hda=fits.PrimaryHDU(f)
# hda.writeto(clndir+'ast-ryan',overwrite=True)

## Vamos modificar o meu astropy para o ryan aqui

In [33]:
#teste para o astropy parecer o Ryan

for k in range(0,axs2,cut_col):
    for l in range(0,axs1,cut_line):
        img = bigimg[l:l+cut_line,k:k+cut_col]
        
        #astropy routine - uses the median to define the center - slower:
        mean, sky, sig = sigma_clipped_stats(img, sigma=2.5)
        background[l:l+cut_line,k:k+cut_col] = sky
        big_sig[l:l+cut_line,k:k+cut_col] = sig
            
        for m in range(0,cut_col,cut_mini_col):
            for n in range(0,cut_line,cut_mini_line):
                
                #clocar condições para fazer as matrizes que passarei o sigma clipp
                il = numpy.amax([jj-lop,0])
                ih = numpy.amin([jj+lop, bxs-1])
                jl = numpy.amax([kk-lop, 0])
                jh = numpy.amin([kk+lop, bxs-1])
                c = img[jl:jh, il:ih]
                
                mini_img = bigimg[n:n+cut_mini_line,m:m+cut_mini_col]
                print(np.shape(mini_img))

                #astropy routine - uses the median to define the center - slower:
                mmean, msky, msig = sigma_clipped_stats(mini_img, sigma = 2.5)
                mini_back[n:n+cut_mini_line,m:m+cut_mini_col] = msky

sub = bigimg - mini_back + background

# finme = "astropy.fits"       
# shd = fits.PrimaryHDU(sub, header=header)
# shd.writeto(clndir+finme,overwrite=True)

(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(64, 55)
(

In [ ]:
# def dif_div(x,y):
#     b = min(x,y)
#     a = max(x,y)
#     if a%b ==0:
#         c = b-1
#         while b % c != 0:
#             c = c-1
#             if x == a:
#                 return b,a
#             else:
#                 return a,b
#     else:
#         while a % b != 0:
#             b=b-1
#         c = min(x,y)
#         d=c-1
#         while c % d != 0:
#             d = d-1
#         #print('maior divisao de ',a,'eh ', b, "e de ", c, "eh", d)
#         if x == a:
#             return b,d
#         else:
#             return d,b
# #     else:
# #         print("There is an mdc but the numbers are")
# #         if x == a:
# #             return b,d
# #         else:
# #             return d,b}

# def max_div(x,y):
#     while mdc(x,y)!=1:
#         x,y = grid(x,y)
#     return x,y


# def loco(x,y):
#     a = max(x,y)
#     b = min(x,y)
#     list_x_div = []
#     list_y_div = []
#     for i in range(1,b+1):
#         c = b%i
#         d = a%(b/i)
#         if c == 0:
#             if a%(b/i)== 0:
#                 list_x_div.append(i)
#                 list_y_div.append(a/(b/i))
#     return list_x_div,list_y_div